In [2]:
from PreProcess.navigate import get_data, open_dat_file, crop_data, channel_outlier_marker
from PreProcess.utils.plotting import figure_compare
from PreProcess.mt_filter import line_filter
import os.path as op
import mne

# %% Set up logging
mne.set_log_file("output.log",
                "%(levelname)s: %(message)s - %(asctime)s",
                overwrite=True)
mne.set_log_level("INFO")

In [3]:
# %% Load the data
TASK = "SentenceRep"
sub_num = 29
HOME = op.expanduser("~")
LAB_root = op.join(HOME, "Box", "CoganLab")
layout, raw, D_dat_raw, D_dat_filt = get_data(29, "SentenceRep", lab_root=LAB_root)
# raw_dat = open_dat_file(D_dat_raw, raw.copy().ch_names)
# dat = open_dat_file(D_dat_filt, raw.copy().ch_names)

C:\Users\ae166\git\IEEG_Pipelines\PreProcess\navigate.py:94: RuntimeWarning: participants.tsv file not found for C:\Users\ae166\Box\CoganLab\BIDS-1.0_SentenceRep\BIDS\sub-D0029\ieeg\sub-D0029_task-SentenceRep_acq-01_run-01_ieeg.edf
  new_raw = read_raw_bids(bids_path=BIDS_path)
C:\Users\ae166\git\IEEG_Pipelines\PreProcess\navigate.py:94: RuntimeWarning: participants.tsv file not found for C:\Users\ae166\Box\CoganLab\BIDS-1.0_SentenceRep\BIDS\sub-D0029\ieeg\sub-D0029_task-SentenceRep_acq-01_run-02_ieeg.edf
  new_raw = read_raw_bids(bids_path=BIDS_path)
C:\Users\ae166\git\IEEG_Pipelines\PreProcess\navigate.py:94: RuntimeWarning: participants.tsv file not found for C:\Users\ae166\Box\CoganLab\BIDS-1.0_SentenceRep\BIDS\sub-D0029\ieeg\sub-D0029_task-SentenceRep_acq-01_run-03_ieeg.edf
  new_raw = read_raw_bids(bids_path=BIDS_path)
C:\Users\ae166\git\IEEG_Pipelines\PreProcess\navigate.py:94: RuntimeWarning: participants.tsv file not found for C:\Users\ae166\Box\CoganLab\BIDS-1.0_SentenceRep\B

In [3]:
# %% Filter the data
# raw.drop_channels(raw.ch_names[10:158])
# raw_dat.drop_channels(raw_dat.ch_names[10:158])
# dat.drop_channels(dat.ch_names[10:158])
# filt = mne.io.read_raw_fif(layout.root + "/derivatives/sub-D00" + str(
#         sub_num) + "_" + TASK + "_filt_ieeg.fif")
filt = line_filter(raw, mt_bandwidth=10.0, n_jobs=-1,
                    filter_length='700ms', verbose=10,
                    freqs=[60], notch_widths=20)
filt2 = line_filter(filt, mt_bandwidth=10.0, n_jobs=-1,
                filter_length='20s', verbose=10,
                freqs=[120, 180, 240], notch_widths=20)


channels:   7%|▋         | 10/140 [03:28<45:13, 20.87s/it]


KeyboardInterrupt: 

In [ ]:
data = [raw, filt, filt2, raw_dat, dat]
figure_compare(data, ["BIDS Un", "BIDS 700ms ", "BIDS 20s+700ms ", "Un",
                    ""], avg=True, verbose=10, proj=True, fmax=250)
figure_compare(data, ["BIDS Un", "BIDS 700ms ", "BIDS 20s+700ms ", "Un",
                    ""], avg=False, verbose=10, proj=True, fmax=250)

In [5]:
# load the data
filt = mne.io.read_raw_fif(layout.root + "/derivatives/sub-D00" + str(29) + "_" + "SentenceRep" + "_filt_ieeg.fif")

In [6]:
# Crop raw data to minimize processing time
new = crop_data(filt)

# Mark channel outliers as bad
channel_outlier_marker(new)

# Exclude bad channels
good = new.copy().drop_channels(new.info['bads'])
good.load_data()

# CAR
good.set_eeg_reference(ref_channels="average", ch_type='seeg')

# Remove intermediates from mem
del new

In [7]:
# fix SentenceRep events
is_sent = False
annot = None
for event in good.annotations:
    if event['description'] in ['Audio']:
        if event['duration'] > 1:
            is_sent = True
        else:
            is_sent = False
    if event['description'] not in [ 'Listen', ':=:' ]:
        if is_sent:
            trial_type = "Sentence/"
        else:
            trial_type = "Word/"
        event['description'] = trial_type + event['description']
    if annot is None:
        annot = mne.Annotations(**event)
    else:
        event.pop('orig_time')
        annot.append(**event)
good.set_annotations(annot)
events, event_id = mne.events_from_annotations(good)
events = mne.merge_events(events, [1, 2], 14, replace_events=False)
event_id['Start'] = 14
good.set_annotations(mne.annotations_from_events(events, good.info['sfreq'], event_id))

TypeError: Event names must be an instance of str or None, got <class 'int'> instead.